In [ ]:
'''Adapted from https://github.com/chrisvdweth/nus-cs4248x/blob/master/3-neural-nlp/Section%204.2%20-%20Transformer%20Machine%20Translation.ipynb'''
import math
from tqdm.notebook import tqdm
from typing import Iterable, List
from timeit import default_timer as timer
import torch
import torch.nn as nn
import pandas as pd
from torch import Tensor
from torch.nn import Transformer

from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.datasets import multi30k, Multi30k

In [ ]:
use_cuda = torch.cuda.is_available()

# Use this line below to enforce the use of the CPU
#use_cuda = False

DEVICE = torch.device("cuda:0" if use_cuda else "cpu")

print("Available device: {}".format(DEVICE))

Available device: cuda:0


In [ ]:
!pip install -U spacy
!python -m spacy download en_core_web_sm
!python -m spacy download zh_core_web_sm
!pip install -U torchdata

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 54.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 MB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 18.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('zh_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 13.4 MB/s eta 0:00:00
     ━

In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.9 MB/s eta 0:00:00


In [ ]:
!pip install portalocker

In [ ]:
from datasets import load_dataset
train_dataset = load_dataset(
                  "ngxingyu/iwslt17_google_trans_scores_sentiments",
                  # streaming=True,
                  split="train",
                )
test_dataset = load_dataset(
                  "ngxingyu/iwslt17_google_trans_scores_sentiments",
                  # streaming=True,
                  split="test",
                )
validation_dataset = load_dataset(
                    "ngxingyu/iwslt17_google_trans_scores_sentiments",
                    # streaming=True,
                    split="validation",
                )
print(train_dataset[0])
print(test_dataset[0])
print(validation_dataset[0])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/229736 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/875 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/8549 [00:00<?, ? examples/s]

{'bleurt_score': 0.5358738303184509, 'comet_score': 0.8854652047157288, 'en': "Thank you so much, Chris. And it's truly a great honor to have the opportunity to come to this stage twice; I'm extremely grateful.", 'google_zh': '非常感谢你，克里斯。非常荣幸有机会两次登上这个舞台；我非常感激。', 'zh': '非常谢谢，克里斯。的确非常荣幸 能有第二次站在这个台上的机会，我真是非常感激。', 'en_sentiment': [-1.896572470664978, -1.776237964630127, 3.2432024478912354], 'zh_sentiment': [-1.788645625114441, -1.4522335529327393, 2.8387067317962646]}
{'bleurt_score': 0.6811590194702148, 'comet_score': 0.9185108542442322, 'en': 'Several years ago here at TED, Peter Skillman  introduced a design challenge  called the marshmallow challenge.', 'google_zh': '几年前，Peter Skillman 在 TED 上提出了一项名为“棉花糖挑战”的设计挑战。', 'zh': '几年前，在TED大会上， Peter Skillman 介绍了一个设计挑战 叫做“棉花糖挑战”', 'en_sentiment': [-3.196884870529175, 1.5497978925704956, 1.2930272817611694], 'zh_sentiment': [-2.8004977703094482, 2.4916672706604004, -0.260890930891037]}
{'bleurt_score': 0.44820863008499146, 'comet_score': 0.8485136

In [ ]:
print(train_dataset, validation_dataset)

Dataset({
    features: ['bleurt_score', 'comet_score', 'en', 'google_zh', 'zh', 'en_sentiment', 'zh_sentiment'],
    num_rows: 229736
}) Dataset({
    features: ['bleurt_score', 'comet_score', 'en', 'google_zh', 'zh', 'en_sentiment', 'zh_sentiment'],
    num_rows: 875
})


In [ ]:
SRC_LANGUAGE = 'en'
TGT_LANGUAGE = 'zh'

# Placeholders
token_transform = {}
vocab_transform = {}

Here we use a **spaCy** tokeniser instead of SPM

In [ ]:
token_transform[SRC_LANGUAGE] = get_tokenizer('spacy', language='en_core_web_sm')
token_transform[TGT_LANGUAGE] = get_tokenizer('spacy', language='zh_core_web_sm')

# Helper function to yield list of tokens
def yield_tokens(data_iter: Iterable, language: str) -> List[str]:
    language_index = {SRC_LANGUAGE: 0, TGT_LANGUAGE: 1}
    for data_sample in data_iter:
        print('here', data_sample[language_index[language]])
        yield token_transform[language](data_sample[language_index[language]])

In [ ]:
from torch.utils.data import Dataset
from typing import Iterable, List

class CustomDataset(Dataset):
    def __init__(self, data_iter: Iterable, token_transform):
        self.data_iter = data_iter
        self.token_transform = token_transform
        self.process_data_iter()

    def __len__(self):
        return len(self.data_iter)

    def __getitem__(self, idx):
        sample = self.data_iter[idx]
        return sample

    def process_data_iter(self):
        en_data = self.data_iter['en']
        zh_data = self.data_iter['zh']
        iter = []
        for i in range(len(self.data_iter)):
          data_sample = {}
          data_sample['en'] = en_data[i]
          data_sample['zh'] = zh_data[i]
          iter.append(data_sample)
        self.data_iter = iter

    def yield_tokens(self, language: str):
        with tqdm(total=len(self.data_iter)) as progress_bar:
            for data_sample in self.data_iter:
                yield self.token_transform[language](data_sample[language])
                progress_bar.update(1)

In [ ]:
from torchtext.datasets import multi30k, Multi30k
custom_train_dataset = CustomDataset(train_dataset, token_transform)
custom_validation_dataset = CustomDataset(validation_dataset, token_transform)
custom_test_dataset = CustomDataset(test_dataset, token_transform)

# Define special symbols and indices
PAD_IDX, UNK_IDX, SOS_IDX, EOS_IDX = 0, 1, 2, 3
# Make sure the tokens are in order of their indices to properly insert them in vocab
special_symbols = ['<PAD>', '<UNK>', '<SOS>', '<EOS>']

for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
    # Create torchtext's Vocab object
    vocab_transform[ln] = build_vocab_from_iterator(custom_train_dataset.yield_tokens(ln),
                                                    min_freq=1,
                                                    specials=special_symbols,
                                                    special_first=True)

# Set `UNK_IDX` as the default index. This index is returned when the token is not found.
# If not set, it throws ``RuntimeError`` when the queried token is not found in the Vocabulary.
for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
    vocab_transform[ln].set_default_index(UNK_IDX)

  0%|          | 0/229736 [00:00<?, ?it/s]

  0%|          | 0/229736 [00:00<?, ?it/s]

In [ ]:
print('EN vocab length:', len(vocab_transform[SRC_LANGUAGE]))
print('ZH vocab length:', len(vocab_transform[TGT_LANGUAGE]))

EN vocab length: 62604
ZH vocab length: 101152


In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self,
                 emb_size: int,
                 dropout: float,
                 maxlen: int = 5000):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2)* math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: Tensor):
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])


class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens: Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

In [ ]:
class Seq2SeqTransformer(nn.Module):
    def __init__(self,
                 num_encoder_layers: int,
                 num_decoder_layers: int,
                 emb_size: int,
                 nhead: int,
                 src_vocab_size: int,
                 tgt_vocab_size: int,
                 dim_feedforward: int = 512,
                 dropout: float = 0.1):
        super(Seq2SeqTransformer, self).__init__()
        self.transformer = Transformer(d_model=emb_size,
                                       nhead=nhead,
                                       num_encoder_layers=num_encoder_layers,
                                       num_decoder_layers=num_decoder_layers,
                                       dim_feedforward=dim_feedforward,
                                       dropout=dropout)
        self.generator = nn.Linear(emb_size, tgt_vocab_size)
        self.src_tok_emb = TokenEmbedding(src_vocab_size, emb_size)
        self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)
        self.positional_encoding = PositionalEncoding(emb_size, dropout=dropout)

    def forward(self,
                src: Tensor,
                trg: Tensor,
                src_mask: Tensor,
                tgt_mask: Tensor,
                src_padding_mask: Tensor,
                tgt_padding_mask: Tensor,
                memory_key_padding_mask: Tensor):
        src_emb = self.positional_encoding(self.src_tok_emb(src))
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
        outs = self.transformer(src_emb, tgt_emb, src_mask, tgt_mask, None, src_padding_mask, tgt_padding_mask, memory_key_padding_mask)
        return self.generator(outs)

    def encode(self, src: Tensor, src_mask: Tensor):
        return self.transformer.encoder(self.positional_encoding(self.src_tok_emb(src)), src_mask)

    def decode(self, tgt: Tensor, memory: Tensor, tgt_mask: Tensor):
        return self.transformer.decoder(self.positional_encoding(self.tgt_tok_emb(tgt)), memory, tgt_mask)

In [ ]:
def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=DEVICE)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask.bool()


def create_mask(src, tgt):
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len),device=DEVICE).type(torch.bool)

    src_padding_mask = (src == PAD_IDX).transpose(0, 1)
    tgt_padding_mask = (tgt == PAD_IDX).transpose(0, 1)

    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

In [ ]:
# Helper method to club together sequential operations
def sequential_transforms(*transforms):
    def func(txt_input):
        for transform in transforms:
            txt_input = transform(txt_input)
        return txt_input
    return func

# method to add SOS/EOS and create tensor for input sequence indices
def tensor_transform(token_ids: List[int]):
    return torch.cat((torch.tensor([SOS_IDX]), torch.tensor(token_ids), torch.tensor([EOS_IDX])))


# `src` and `tgt` language text transforms to convert raw strings into tensors indices
text_transform = {}
for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
    text_transform[ln] = sequential_transforms(token_transform[ln], # Tokenization
                                               vocab_transform[ln], # Vectorization
                                               tensor_transform)    # Add SOS/EOS and create tensor


# Method to collate data samples into batch tensors
def collate_fn(batch):
    src_batch, tgt_batch = [], []
    for i in range(len(batch)):
        src_sample, tgt_sample = batch[i][SRC_LANGUAGE], batch[i][TGT_LANGUAGE]
        src_batch.append(text_transform[SRC_LANGUAGE](src_sample.rstrip("\n")))
        tgt_batch.append(text_transform[TGT_LANGUAGE](tgt_sample.rstrip("\n")))

    src_batch = pad_sequence(src_batch, padding_value=PAD_IDX)
    tgt_batch = pad_sequence(tgt_batch, padding_value=PAD_IDX)
    return src_batch, tgt_batch

In [ ]:
def train_epoch(model, optimizer, criterion):
    model.train()
    losses = 0

    train_dataloader = DataLoader(custom_train_dataset.data_iter, batch_size=BATCH_SIZE, collate_fn=collate_fn)

    for src, tgt in tqdm(train_dataloader, total=len(list(train_dataloader))):
        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)

        # Remove last entry an all target sequences (typically PAD, can be EOS)
        tgt_input = tgt[:-1, :]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

        logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)

        optimizer.zero_grad()

        # Remove <SOS> from all targets
        tgt_out = tgt[1:, :]

        loss = criterion(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        loss.backward()

        optimizer.step()
        losses += loss.item()

    return losses / len(list(train_dataloader))


def evaluate(model):
    model.eval()
    losses = 0

    val_dataloader = DataLoader(custom_validation_dataset.data_iter, batch_size=BATCH_SIZE, collate_fn=collate_fn)

    for src, tgt in tqdm(val_dataloader, total=len(list(val_dataloader))):
        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)

        tgt_input = tgt[:-1, :]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

        logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)

        tgt_out = tgt[1:, :]

        loss = criterion(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        losses += loss.item()

    return losses / len(list(val_dataloader))

In [ ]:
torch.manual_seed(0)

SRC_VOCAB_SIZE = len(vocab_transform[SRC_LANGUAGE])
TGT_VOCAB_SIZE = len(vocab_transform[TGT_LANGUAGE])
EMB_SIZE = 512
NHEAD = 8
FFN_HID_DIM = 512
BATCH_SIZE = 8
NUM_ENCODER_LAYERS = 3
NUM_DECODER_LAYERS = 3

# Create model
transformer = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE, NHEAD, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, FFN_HID_DIM)

# Initialize weights
for p in transformer.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

# Move model to device (ideally GPU, otherwise CPU)
transformer = transformer.to(DEVICE)

# Define loss function
criterion = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX)

# Define optimizer
optimizer = torch.optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

In [ ]:
NUM_EPOCHS = 2

for epoch in range(1, NUM_EPOCHS+1):
    start_time = timer()
    train_loss = train_epoch(transformer, optimizer, criterion)
    end_time = timer()
    val_loss = evaluate(transformer)
    print((f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}, "f"Epoch time (total) = {(end_time - start_time):.3f}s"))

  0%|          | 0/28717 [00:00<?, ?it/s]

  0%|          | 0/110 [00:00<?, ?it/s]

Epoch: 1, Train loss: 5.999, Val loss: 6.148, Epoch time (total) = 1980.463s


  0%|          | 0/28717 [00:00<?, ?it/s]

  0%|          | 0/110 [00:00<?, ?it/s]

Epoch: 2, Train loss: 5.505, Val loss: 5.891, Epoch time (total) = 1944.229s


In [ ]:
# Method to generate output sequence using greedy algorithm
def greedy_decode(model, src, src_mask, max_len, start_symbol):
    src = src.to(DEVICE)
    src_mask = src_mask.to(DEVICE)

    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(DEVICE)
    for i in range(max_len-1):
        memory = memory.to(DEVICE)
        tgt_mask = (generate_square_subsequent_mask(ys.size(0)).type(torch.bool)).to(DEVICE)
        out = model.decode(ys, memory, tgt_mask)
        out = out.transpose(0, 1)
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim=1)
        next_word = next_word.item()

        ys = torch.cat([ys, torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=0)
        if next_word == EOS_IDX:
            break
    return ys

# Actual method to translate input sentence into target language
def translate(model: torch.nn.Module, src_sentence: str):
    model.eval()
    src = text_transform[SRC_LANGUAGE](src_sentence).view(-1, 1)
    num_tokens = src.shape[0]
    src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)
    tgt_tokens = greedy_decode(model, src, src_mask, max_len=num_tokens+5, start_symbol=SOS_IDX).flatten()

    return " ".join(vocab_transform[TGT_LANGUAGE].lookup_tokens(list(tgt_tokens.cpu().numpy()))).replace("<SOS>", "").replace("<EOS>", "")

def translate_dataset(model, validation_dataset):
    translations = []
    for sample in tqdm(validation_dataset, total=len(validation_dataset)):
        src_sentence = sample[SRC_LANGUAGE]
        tgt_sentence = sample[TGT_LANGUAGE]

        # Translate the source sentence using the model
        translated_sentence = translate(model, src_sentence)

        # Append the translation along with source and target sentences
        translations.append({'en': src_sentence, 'zh': tgt_sentence, 'translated_output': translated_sentence})

    return pd.DataFrame(translations)

In [ ]:
print(translate(transformer, "Thank you very much."))
print(translate(transformer, "And it was a huge success."))

 谢谢 你 的 生活 
 这是 一个 很 大 的 问题 


In [ ]:
translations = translate_dataset(transformer, custom_test_dataset.data_iter)
translations.to_csv('transformer_translations.csv', index=True)

  0%|          | 0/8549 [00:00<?, ?it/s]

In [ ]:
translations.head()

,en,zh,translated_output
0,"Several years ago here at TED, Peter Skillman ...",几年前，在TED大会上， Peter Skillman 介绍了一个设计挑战 叫做“棉花糖挑战”,在 这个 世界 上 的 一 种 “ 我们 可以 在 一个 更 多 的 地方 中 找到 一个...
1,And the idea's pretty simple: Teams of four h...,是个非常简单的主意 要求一组四人的团队搭建一个独立的最高建筑 材料是20根意大利面条 一段胶...,现在 ， 这个 想法 是 ： “ 这是 一个 很 好的 地方 ， 这 一 种 非常 重要 ...
2,The marshmallow has to be on top.,棉花糖必须放在最上面,♫
3,"And, though it seems really simple, it's actua...",这虽然看似简单，其实并不容易 因为它要求人们 迅速地合作,所以 ， 如果 你 想 做 的 是 ： “ 如果 你 有 一个 更 多 的 人 ， 那 就...
4,"And so, I thought this was an interesting idea...",我觉得这是个有趣的主意 我把它放到了设计专题讨论会上,所以 我 想 说 的 是 ： “ 我 是 一个 很 好的 想法 ， 我 想 让 大家 更 ...
